# Geöffnete Skilifte absaugen



To Do:
- Crontab einrichten (Digital Ocean?)
- Fail-Benachrichtigung
- Zusätzliches Script für Analyse der Skigebiete: Anzahl Lifte, Anzahl Pistenkilometer

Fragen:
- Hilfe bei Digital Ocean
- Was mache ich in dem halben Jahr, in dem der Code läuft? (Zwischenauswertungen??)

In [10]:
# Libraries laden
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Teil 1: Liste mit allen Live-Skigebieten finden

In [11]:
startseite = requests.get("https://www.bergfex.ch/schweiz/").text
soup_startseite = BeautifulSoup(startseite, "html.parser")

Die Links heraussuchen

In [12]:
liveskigebiete = soup_startseite.find_all('li', {'style':'font-weight: bold;'})
linkliste = []

for gebiet in liveskigebiete:
    link = gebiet.find('a')['href']
    linkliste.append(link)

In [13]:
pd.DataFrame(linkliste).to_csv('skigebiete_linkliste.csv', index=False)

## Teil 2: Öffnungszeiten auslesen
Leere Liste starten

In [5]:
skigebiete_final = []

Seiten absaugen & souppieren

In [6]:
for gebiet_link in linkliste:
    website = requests.get("https://www.bergfex.ch"+gebiet_link).text
    soup = BeautifulSoup(website, "html.parser")

    # Titel des Skigebiets
    titel = soup.find('h1', {"class": "has-sup"})
    titel = (titel.text.replace("Skigebiet ", ""))
    # Offene Skilifte auslesen
    for skigebiet in soup.find_all('table', class_="schneewerte"):
        try:
            offenelifte = (skigebiet.find_all('td')[-1].text).split(" von ") # Auslesen im Format "12 von 24" (offene Lifte sind immer das letzte TD, deshalb -1)
            total = offenelifte[1].strip()
            offen = offenelifte[0].strip()
            skigebiete_final.append({"titel": titel, "totallifte": total, "offen": offen}) # In die finale Liste laden
        except:
            print("Fehler mit Skigebiet: "+"https://www.bergfex.ch"+gebiet_link)



## Ergänzen in einer Tabelle

In [7]:
df_skigebiete = pd.DataFrame(skigebiete_final)

In [8]:
import datetime

now = datetime.datetime.now()
#print (now.strftime("%Y-%m-%d %H:%M"))
df_skigebiete.to_csv('skigebiete_oeffnungszeiten/'+now.strftime("%Y-%m-%d_%H-%M_skigebiete_oeffnungszeiten.csv"))

## Teil 3: Alarmieren, wenn irgendetwas fehlschlägt

In [9]:
# Send E-Mail